# Code & example to update refineGEMs' internal database
The following code transforms a human-readable TSV file in the FILE_DIRECTORY (I) into the data format required to update the internal database of refineGEMs (II & III).

In [1]:
from refinegems.classes.medium import update_db_multi, updated_db_to_schema
import pandas as pd
import numpy as np
import ntpath
import os

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


## (I) File directory
The file directory needs to be specified here. All files from this directory will be used to generate new TSV files compatible with the database.

In [2]:
FILE_DIRECTORY = './oils_for_artificial_sebum/' #'../../../'

## (II) Transformation of human-readable TSV files to database-readable files
The following function `transform_medium_tsv_table_for_update` does the main job in transforming the TSV files into database-readable TSV files. </br>
This function is currently only setting up the input for the database tables 'medium2substance' and 'substance2db'. </br>
The code cell underneath the function definition cell specifies that only files ending with '_substances.tsv' should be transformed.

In [2]:
def transform_medium_tsv_table_for_update(table_path: str, file_directory: str):
   """Transforms a TSV table containing a medium/subset definition into (a) TSV file(s) usable 
   with the medium.py setup to be used to automatically update the database

   Args:
       - table_path (str): Path to the TSV file defining a medium/ subset
       - file_directory (str): Path to the current workspace directory
   """
   table_path = f'{file_directory}{table_path}'
   # Get medium name & DataFrame
   medium_or_subset_name_list = ntpath.basename(table_path.replace('.tsv', '')).split('_')
   medium_or_subset_name = medium_or_subset_name_list[0]
   medium_or_subset = medium_or_subset_name_list[1]
   if not medium_or_subset in ['medium', 'subset']:
      print('No  medium/subset file was provided. At least no medium/subset was specified in the file name.')
      return
   
   # Get all necessary names from file name
   table_name = 'medium2substance' if medium_or_subset == 'medium' else 'subset2substance'
   flux_or_percent = 'flux' if medium_or_subset == 'medium' else 'percent'
   mos_df = pd.read_csv(table_path, sep='\t')
   
   # Rename relevant & drop unnecessary columns
   mos_df.drop('formula', axis=1, inplace=True)
   mos_df.rename({'name': 'substance'}, axis=1, inplace=True)
   
   # Add new columns medium_or_subset & 'table' to mos table
   mos2s_df = mos_df.copy() # Copy dataframe to avoid trouble with substance table update
   mos2s_df[medium_or_subset] = medium_or_subset_name
   mos2s_df['table'] = table_name
   
   # Add conditions column to mos table
   mos2s_df['conditions'] = mos2s_df.apply(lambda row: f'substance={row["substance"]};{medium_or_subset}={row[medium_or_subset]}', axis=1)
   mos2s_df.drop(['substance', medium_or_subset], axis=1, inplace=True)
   
   ### Get all relevant columns to update flux/percent if present
   if not mos2s_df[flux_or_percent].empty:
      mos2s_flux_or_percent_df = mos2s_df[[flux_or_percent, 'table', 'conditions']].copy()
      # Rename new_value to 'new_value'
      mos2s_flux_or_percent_df.rename({flux_or_percent: 'new_value'}, axis=1, inplace=True)
      mos2s_flux_or_percent_df['column'] = flux_or_percent
      # Extract mos2s table
      file_substring = 'm2s' if medium_or_subset == 'medium' else 's2s'
      mos2s_flux_or_percent_df.to_csv(f'{file_directory}{medium_or_subset_name}_{file_substring}_{flux_or_percent}_update.tsv', sep='\t', index=False)
      
   ### Get all relevant columns to update source if present
   if not mos2s_df['source'].empty:
      mos2s_source_df = mos2s_df[['source', 'table', 'conditions']].copy()
      # Rename 'source' to 'new_value'
      mos2s_source_df.rename({'source': 'new_value'}, axis=1, inplace=True)
      mos2s_source_df['column'] = 'source'
      # Extract mos2s table
      file_substring = 'm2s' if medium_or_subset == 'medium' else 's2s'
      mos2s_source_df.to_csv(f'{file_directory}{medium_or_subset_name}_{file_substring}_source_update.tsv', sep='\t', index=False)
      
   if mos2s_df[flux_or_percent].empty or mos2s_df['source'].empty: print('No updatable column for a medium/subset update present.')
   
   if len(mos_df.columns) > 4:
      ### Get the s2db table for update, if possible
      s2db_df = mos_df.drop([flux_or_percent,'source'], axis=1)
      
      # Merge VMH & BiGG if they have the same ID & Column does not already exist
      if not 'BiGG+VMH' in s2db_df.columns:
         # Create new column 'BiGG+VMH'
         s2db_df['BiGG+VMH'] = np.NaN
         
         # Merge VMH & BiGG column for same IDs & Remove original entries
         def merge_BiGG_VMH(row: pd.Series):
            if (row['BiGG'] == (row['VMH'])): # | (row["BiGG"].isna() & row["VMH"].isna())
               row['BiGG+VMH'] = row['BiGG']
               row['BiGG'] = np.NaN # Remove entry from column
               row['VMH'] = np.NaN # Remove entry from column
            else:
               row['BiGG+VMH'] = np.NaN
            return row
      
         s2db_df = s2db_df.apply(merge_BiGG_VMH, axis=1)
      
      # Transform table into long format
      s2db_df = pd.melt(s2db_df, id_vars='substance', var_name='db_type', value_name='db_id', ignore_index=True)
      
      # Remove all NaNs
      s2db_df.dropna(inplace=True)
      
      # Add new columns table & column to s2db table
      s2db_df['table'] = 'substance2db'
      s2db_df['column'] = 'substance_id, db_id, db_type'
      
      # Create 'new_value' from 'db_type' & 'db_id'
      s2db_df['new_value'] = s2db_df.apply(lambda row: f'{row["db_id"]}, {row["db_type"]}', axis=1)
      s2db_df.drop(['db_type', 'db_id'], axis=1, inplace=True)
      
      # Add conditions column to s2db table
      s2db_df['conditions'] = s2db_df.apply(lambda row: f'substance={row["substance"]}', axis=1)
      s2db_df.drop('substance', axis=1, inplace=True)
      
      # Extract s2db table
      s2db_df.to_csv(f'{file_directory}{medium_or_subset_name}_s2db_update.tsv', sep='\t', index=False)

In [6]:
for files in os.listdir(FILE_DIRECTORY):
    if files.endswith('_substances.tsv'):
        if 'already' in files: continue
        print(files)
        transform_medium_tsv_tables_for_update(f'{FILE_DIRECTORY}{files}', FILE_DIRECTORY)
    else:
        continue

RPMI_substances.tsv
CasA_subset_substances.tsv
CGXII_substances.tsv
dGMM_substances.tsv
M9_substances.tsv
CasA_medium_substances.tsv
MP-AU_substances.tsv
SNM3_substances.tsv
LB_substances.tsv


## (III) Add data from the database-readable TSV file to the database
The following code iterates over all newly generated TSV files and updates the database tables 'medium2substance' and 'substance2db' accordingly.

In [4]:
for files in os.listdir(FILE_DIRECTORY):
    if files.endswith('_update.tsv'):
            print(files)
            update = True if 'm2s' in files else False
            df = pd.read_csv(files, sep='\t')
            update_db_multi(df, update_entries=update)
    else:
        continue

CasA_subset_substances_for_s2db_update.tsv
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value=cpd00395, SEED, condition=substance=L-Cysteate
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value=MNXM713, MetaNetX, condition=substance=L-Cysteate
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value=Lcyst, BiGG+VMH, condition=substance=L-Cysteate
CasA_medium_substances_for_m2s_update.tsv
RPMI_substances_for_m2s_update.tsv
SNM3_substances_for_m2s_update.tsv
MP-AU_substances_for_m2s_update.tsv
CGXII_substances_for_m2s_update.tsv
M9_substances_for_m2s_update.tsv
CasA_subset_substances_for_m2s_update.tsv
CGXII_substances_for_s2db_update.tsv
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_

## Update Schema with updated database
!Be careful to  check the changes between the current SQL Schema file and the new one!

In [2]:
updated_db_to_schema()

In [30]:
transform_medium_tsv_tables_for_update('artSe_subset.tsv', FILE_DIRECTORY)

In [5]:
s2s_percent_df = pd.read_csv('./oils_for_artificial_sebum/artSe_s2s_percent_update.tsv', sep='\t')
update_db_multi(s2s_percent_df, update_entries=False)
#s2s_source_df = pd.read_csv('./oils_for_artificial_sebum/artSe_s2s_source_update.tsv')
substances_df = pd.read_csv('./oils_for_artificial_sebum/artSe_s2db_update.tsv', sep='\t')
update_db_multi(substances_df, update_entries=False)

UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value= chsterol, BiGG, condition=substance=Cholesterol
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value= ocdcea, BiGG, condition=substance=Octadecenoate [Oleic acid]
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value= ttdca, BiGG, condition=substance=Tetradecanoate
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value= ddca, BiGG, condition=substance=Dodecanoate
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value= hdca, BiGG, condition=substance=Hexadecanoate [Palmitic acid]
UNIQUE constraint failed: substance2db.substance_id, substance2db.d

In [5]:
def transform_substance2db_tsv_table_for_update(table_path: str, file_directory: str):
   """Transforms a TSV table containing a medium/subset definition into (a) TSV file(s) usable 
   with the medium.py setup to be used to automatically update the database

   Args:
       - table_path (str): Path to the TSV file defining a medium/ subset
       - file_directory (str): Path to the current workspace directory
   """
   table_path = f'{file_directory}{table_path}'
   s2db_df = pd.read_csv(table_path, sep='\t')

   # Merge VMH & BiGG if they have the same ID & Column does not already exist
   if not 'BiGG+VMH' in s2db_df.columns:
      # Create new column 'BiGG+VMH'
      s2db_df['BiGG+VMH'] = np.NaN
      # Merge VMH & BiGG column for same IDs & Remove original entries
      def merge_BiGG_VMH(row: pd.Series):
         if (row['BiGG'] == (row['VMH'])): # | (row["BiGG"].isna() & row["VMH"].isna())
            row['BiGG+VMH'] = row['BiGG']
            row['BiGG'] = np.NaN # Remove entry from column
            row['VMH'] = np.NaN # Remove entry from column
         else:
            row['BiGG+VMH'] = np.NaN
         return row
      s2db_df = s2db_df.apply(merge_BiGG_VMH, axis=1)

   # Transform table into long format
   s2db_df = pd.melt(s2db_df, id_vars='substance', var_name='db_type', value_name='db_id', ignore_index=True)
   
   # Remove all NaNs
   s2db_df.dropna(inplace=True)
   
   # Add new columns table & column to s2db table
   s2db_df['table'] = 'substance2db'
   s2db_df['column'] = 'substance_id, db_id, db_type'
   
   # Create 'new_value' from 'db_type' & 'db_id'
   s2db_df['new_value'] = s2db_df.apply(lambda row: f'{row["db_id"]}, {row["db_type"]}', axis=1)
   s2db_df.drop(['db_type', 'db_id'], axis=1, inplace=True)
   
   # Add conditions column to s2db table
   s2db_df['conditions'] = s2db_df.apply(lambda row: f'substance={row["substance"]}', axis=1)
   s2db_df.drop('substance', axis=1, inplace=True)
   
   # Extract s2db table
   s2db_df.to_csv(f'{file_directory}{ntpath.basename(table_path.replace(".tsv", ""))}_s2db_update.tsv', sep='\t', index=False)

In [6]:
transform_substance2db_tsv_table_for_update('SNM3.tsv', '/Users/brune/Documents/10_Code/possible-modules-for-rg/')


In [7]:
substances_df = pd.read_csv('/Users/brune/Documents/10_Code/possible-modules-for-rg/SNM3_s2db_update.tsv', sep='\t')
update_db_multi(substances_df, update_entries=False)



UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value=pnto__R, BiGG, condition=substance=3-[(2R)-2,4-Dihydroxy-3,3-dimethylbutanamido]propanoate [Pantothenate]
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value=glc__D, BiGG, condition=substance=D-Glucose
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value=orn__D, BiGG, condition=substance=D-Ornithine
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value=ala__L, BiGG, condition=substance=L-Alanine
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value=arg__L, BiGG, condition=substance=L-Arginine
UNIQUE constraint failed: substance2db.substance_i

FileNotFoundError: [Errno 2] No such file or directory: './src/refinegems/data/database/updated_media_db.sql'

In [9]:
updated_db_to_schema('/Users/brune/Documents/10_Code/refinegems/src/refinegems/data/database')

In [1]:
from refinegems.classes.medium import update_db_multi, updated_db_to_schema
import pandas as pd

substances_df = pd.read_csv('/Users/brune/Documents/10_Code/possible-modules-for-rg/media-defs/SNM3_s2db_update_test.tsv', sep='\t')
update_db_multi(substances_df, update_entries=False)

['BiGG', 'VMH']
['BiGG', 'VMH']
[]
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value=so4, BiGG+VMH, condition=substance=Sulfate [SO4(2-)]
['BiGG', 'VMH']
['BiGG', 'VMH']
[]
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value=thm, BiGG+VMH, condition=substance=Thiamine
['BiGG', 'VMH']
['BiGG', 'VMH']
[]
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value=urea, BiGG+VMH, condition=substance=Urea
['BiGG', 'VMH']
['BiGG', 'VMH']
[]
UNIQUE constraint failed: substance2db.substance_id, substance2db.db_id
Ocurred with: column=substance_id, db_id, db_type, new_value=h2o, BiGG+VMH, condition=substance=Water [H2O]


/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)
